In [6]:
pip install requests pandas geopandas shapely

In [ ]:
from dataclasses import dataclass
from datetime import datetime
from typing import Optional

import requests
import logging

In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

In [ ]:
DATE_FORMAT = "%Y-%m-%dT00:00:00.000Z"
AUTH_URL  = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
PRODUCTS_URL = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
DOWNLOAD_URL = "https://download.dataspace.copernicus.eu/odata/v1/Products"

In [ ]:
copernicus_username = "21ssds415008@msruas.ac.in"
copernicus_password = "Myresearch@2021"
data_collection = "SENTINEL-2"

In [ ]:
@dataclass()
class Coordinates:
    longitude: str
    latitude: str

    def stringify(self) -> str:
        return f"{self.longitude} {self.latitude}"

In [8]:
def get_access_token(username: str, password: str) -> Optional[str]:
    logging.info(f"Username: {username}, Password: {password}")
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }

    response = request.post(AUTH_URL, data=data)
    if response.status_code != 200:
        return None
    
    json_response = response.json()
    return json_response.get("access_token")

In [ ]:
def get_report(coordinates: Coordinates, start_date: datetime, end_date: datetime) -> Optional[dict]:
    # ft='POLYGON ((57.692102513344906 -20.35226228118077, 57.692102513344906 -20.4745116407249, 57.77699025382094 -20.4745116407249, 57.77699025382094 -20.35226228118077, 57.692102513344906 -20.35226228118077))'
    filters = [
        f"Collection/Name eq '{data_collection}'",
        # f"OData.CSC.Intersects(area=geography'SRID=4326;{polygon}')",
        f"OData.CSC.Intersects(area=geography'SRID=4326;POINT({coordinates.stringify()})')",
        f"ContentDate/Start gt {start_date.strftime(DATE_FORMAT)}",
        f"ContentDate/Start lt {end_date.strftime(DATE_FORMAT)}",
    ]

    response = requests.get(PRODUCTS_URL + f"?$filter={' and '.join(filters)}&$count=True&$top=1000")
    if response.status_code != 200:
        return None
    
    json_data = response.json()
    return json_data.get("value")

In [ ]:
def download_image(token: str, uuid: str) -> None:
    session = requests.Session()
    session.headers.update({"Authorization": f"Bearer {token}"})
    url = DOWNLOAD_URL + f"({uuid})/$value"
    response = session.get(url, allow_redirects=False)

In [ ]:
def process_data(data: dict):
    dataframes = pd.DataFrame.from_dict(data)
    if dataframes.shape[0] > 0:
        dataframes['geometry'] = dataframes["GeoFootprint"].apply(shape)
        geo_dataframes = gpd.GeoDataFrame(dataframes).set_geometry("geometry") # Convert PD to GPD
        geo_dataframes = geo_dataframes[~geo_dataframes["Name"].str.contians("L1C")]
        logging.info(f"Total L2A tiles found {len(geo_dataframes)}")
        geo_dataframes["identifier"] = geo_dataframes["Name"].str.split(".").str[0]
        
        return geo_dataframes

In [ ]:
coordinates = Coordinates(longitude="57.692102513344906", latitude="-20.4745116407249")
start_date = datetime(2020, 9, 10)
end_date = datetime(2020, 9, 20)

In [ ]:
if __name__ == "__main__":
    report = get_report(coordinates, start_date, end_date)
    if report:
        data = process_data(report)
        if data:
            token = get_access_token(copernicus_username, copernicus_password)
            for index, feature in enumerate(data.iterfeatures()):
                download_image(token, feature['properties']['Id'])